Python for Data Analysis, GMC, Vilnius University, 2025

# HW2: Custom functions

**Rules:**
- Complete tasks by writing code from scratch (read the task for more info).
- Do not add any new imports. 💡 Notice that you don't yet need Pandas or other non-native packages.
- Do not modify contents of cells with `assert` statements. If variable names are indicated in the task, use them, or you may get an assertion error.
- Write type hints where possible.
- Do not create classes (you shouldn't know yet what are those).


There are 6 tasks in this Notebook, each costs 0.667 ($\frac{2}{3}$) points. You need to collect 4 points to get the maximum grade.


**Each task consists of:**
1. Text cell containing the task description.
1. Code cell to solve the task.
1. (Sometimes) Code cell with `assert` statements. Run it after providing your solution to ensure you didn't accidentally change or break something.

Don't hesitate to contact me or Martynas if you are stuck.

In [1]:
import math, string, urllib
from tqdm import tqdm   # you can use it to add progress bars if you have slow loops

# 1 📐

The code below calculates rounded RMSD between two lists of numbers (`values0` and `values1`). The result is stored in `round_rmsd`:

```python
values0 = [0.1, 1.2, 4.1, 7.7, 2.0, 8.2, 3.4]
values1 = [9.1, 1.2, 4.4, 7.6, 2.1, 8.8, 6.7]
differences = []

for i in range(len(values0)):
    d = abs(values0[i] - values1[i])
    differences.append(d ** 2)

msd = sum(differences) / len(differences)
rmsd = msd ** 0.5
round_rmsd = round(rmsd, 2)
```

Convert this code into a function named `get_rmsd()`. It should have 2 parameters (two lists of numbers). It should return one value (rounded RMSD). Add an `if-else` statement into function to ensure that lists of given values are not empty and have the same length, otherwise return `None` (do not raise any errors).

In [54]:
# your code
def get_rmsd(list_1: list[float | int], list_2: list[float | int]) -> float | None:

    if((not list_1 or not list_2 )
        or len(list_1) != len(list_2)) :
        return None

    try:
        differences = [abs(x_i - x_j) ** 2 for x_i, x_j in zip(list_1, list_2)]
        msd = sum(differences) / len(differences)
        rmsd = msd ** 0.5
        round_rmd = round(rmsd, 2)
        return round_rmd
    except Exception:
        return None


values0 = [0.1, 1.2, 4.1, 7.7, 2.0, 8.2, 3.4]
values1 = [9.1, 1.2, 4.4, 7.6, 2.1, 8.8, 6.7]
rmsd = get_rmsd(values0, values1)
print(rmsd)

3.63


In [56]:
assert get_rmsd([0.1, 1.2, 4.1, 7.7, 2.0, 8.2, 3.4],
                [9.1, 1.2, 4.4, 7.6, 2.1, 8.8, 6.7]) == 3.63
assert get_rmsd([1], [3]) == 2.0

assert get_rmsd([], []) == None
assert get_rmsd([1, 2, 3], [1, 2]) == None  # lists of unequal lengths

# 2 👽
You are an [exobiology](https://en.wikipedia.org/wiki/Astrobiology) assistant. Your supervisor has sent you some sequences of extraterrestrial bacteria. These bacteria use a different genetic code, noted by X, Z, W, Y, and Q letters for its bases, and has these transcription rules:
- X is complementary to Z;
- W is complementary to Y;
- The extra rare base Q is always turned into W during the transcription process.

Write a function named `transcribe()` which transcribes extraterrestial bacteria sequences. Ensure the function is not case-sensitive, and warns the user if the sequence contains incorrect bases. Add an option (not a requirement) to provide custom transcription rules (e.g. human ones) as a second parameter.

In [57]:
# your code
def transcribe(seq: str, rules: dict[str, str] = None) -> str | None:
    if rules is None:
        rules = {"X": "Z",
                 "Z": "X",
                 "W": "Y",
                 "Y": "W",
                 "Q": "W"}

    allowed = set(rules.keys())
    seq_upper = seq.upper()
    if not set(seq_upper).issubset(allowed):
        print('Found incorrect bases!')
        return None

    rna_seq = [rules[char] for char in seq_upper]
    return "".join(rna_seq)

In [61]:
assert transcribe("ZYWXQYQXWYXZXZXZYXWYZXQYWWXXXZYZYXWYXZ") == "XWYZWWWZYWZXZXZXWZYWXZWWYYZZZXWXWZYWZX"
assert transcribe("YqQyyQ") == "WWWWWW"

# 3 📻

You've intercepted a coded message where each letter is shifted by a certain number in the alphabet ([Caesar cipher](https://en.wikipedia.org/wiki/Caesar_cipher)). You know that the cipher's alphabet is all English lowercase letters and the space character. Write a function `decode_caesar()` that takes a message and a shift value, then returns the decoded text.
*   Do not write yourself all possible letters - get them from the `string` package (you can start searching how to do that [here](https://www.w3schools.com/Python/ref_module_string.asp)).
*   Assert that the input only contains allowed symbols.

In [62]:
# your code
def decode_caesar(encoded_message: str, shift: int) -> str:
    decoded_message = ""
    alias = string.ascii_lowercase + " "
    assert set(encoded_message).issubset(set(alias)), 'message contains unsuported letters'
    pos = { c:idx for idx, c in enumerate(alias) }
    shift = shift % len(alias)
    for char in encoded_message:
        idx = ( pos[char] - shift ) % len(alias)
        decoded_message += alias[idx]
    return decoded_message

In [63]:
assert decode_caesar('ltlynlujegmyvngzp ah pvu', 7) == 'emergency frog situation'
assert decode_caesar('uncoded message', 0) == 'uncoded message'
assert decode_caesar('full circle', 27) == 'full circle'

# 4 🕒

Write a meeting scheduler which would find common free time between N people. Name the function `find_available_slots()`.

The function should take:
1. A limit from which to which hour a meeting can be made (e.g. `(9, 17)` to indicate it is from 9:00 to 17:00).
1. an arbitrary number of lists, where each list contains tuples of busy hours. E.g. `[(9, 11), (13, 14)]` means that this person is busy from 9:00 to 11:00, and then busy for one hour from 13:00 to 14:00.

The function should return a list of all hours when **all** people are not busy. Return hours as numbers (see examples in the assert statements).

- If a single list was provided, the program should just return free time of that single person.
- If an empty list was provided, it is assumed that person is free the whole day.
- Raise an error of your choice if an incorrect hour value is given somewhere in the input (e.g. 25 or "bla"); or the second hour indicated in the tuple is smaller than the first.

In [64]:
# your code
def convert_time_to_list(time_range: tuple[int, int]) -> list[int]:
    try:
        if time_range[1] < time_range[0]:
            raise ValueError(f"Time paradox detected! Please check your time ranges {time_range=} or contact the Bronze Dragonflight for assistance.")
        if time_range[1] // 24 or time_range[0] // 24:
            raise ValueError(f"Impossible time usage! {time_range=}")
        return [hour for hour in range(time_range[0], time_range[1])]
    except TypeError:
        raise TypeError("Time ranges must be integers!")
    except IndexError:
        raise IndexError(f"Loss of value in time ranges: {time_range=}")

def find_available_slots(
        time_range: tuple[int, int],
        *args: list[tuple[int, int]]
    ) -> list[int]:
    available_hours = set(convert_time_to_list(time_range))
    busy_hours = {
        hour
        for busy_list in args
        for busy_tuple in busy_list
        for hour in convert_time_to_list(busy_tuple)
    }
    return sorted(available_hours - busy_hours)

In [65]:
assert find_available_slots((9, 17), [(9, 10), (16, 18)], [(12, 15)]) == [10, 11, 15]
assert find_available_slots((14, 17), [(8, 10), (16, 18)]) == [14, 15]
assert find_available_slots((14, 17), []) == [14, 15, 16]
assert find_available_slots((14, 17), [(14, 15)], [(15, 17)]) == []

# 5 🎭

The [PAD emotional state model](https://en.wikipedia.org/wiki/PAD_emotional_state_model) is a psychological model to describe and measure emotional states, and uses three numerical dimensions: Pleasure (P), Arousal (A), and Dominance (D).

Download the file with `urllib` using the provided link. The file contains records about emotional states of several subjects, using the PAD emotional state model. Each subject noted their pleasure, arousal, and dominance level on a scale from 1 (lowest) to 10 (highest).
1. Validate that all emotion values are in the correct range. Print out a warning message for any out-of-range numbers (do not raise exceptions).
1. Print out how many subjects were excited (all three dimensions > 6).
1. Find a subject(s) who's emotion is the most similar to subject's named Sandra Thompson (the first one in the list). Print out the name of that subject (or those subjects, if several). To measure the distance between emotions, consider the emotion numbers to be coordinates, and use Euclidean distance $\sqrt{(x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2}$.

In [66]:
link = "https://raw.githubusercontent.com/Tallivm/vu-python/refs/heads/main/emotions.csv"
# your code

def validate_data(
        data: list[tuple[str, str, int, int, int]]
    ) -> None:
    for line in data:
        if not all(1 <= value <= 10 for value in line[2:]):
            print(f"Values in line are out of range: {line}")

def calculate_excited(
        data: list[tuple[str, str, int, int, int]]
    ) -> int:
    excited_counter = 0
    for line in data:
        if all(value > 6 for value in line[2:]):
            excited_counter += 1
    return excited_counter

def count_euclidean_distance(
        line_1: tuple[str, str, int, int, int],
        line_2: tuple[str, str, int, int, int]
    ) -> float:
    return math.sqrt(sum(
        (x_i - x_j)**2
        for x_i, x_j in zip(line_1[2:], line_2[2:])
    ))

file_name = "emotions.csv"
urllib.request.urlretrieve(link, file_name)
with open(file_name, 'r') as fh:
    content = fh.readlines()

data = [
    (name, surname, int(p), int(a), int(d))
    for name, surname, p, a, d in
    (line.strip().split() for line in content)
]
validate_data(data)
print(f"Excited people count: {calculate_excited(data)}")

sandra_line = data[0]
min_dist = None
most_similar = []
for line in data[1:]:
    current_dist = count_euclidean_distance(sandra_line, line)
    if min_dist is None or min_dist > current_dist:
        min_dist = current_dist
        most_similar = []

    if min_dist == current_dist:
        most_similar.append(line)
names = ", ".join(f"{n} {s}" for n, s, *_ in most_similar)
print(f"Most similar to Sandra Thompson: {names}")

Values in line are out of range: ('William', 'Lewis', 7, 6, 0)
Values in line are out of range: ('Joshua', 'Jackson', 0, 7, 3)
Values in line are out of range: ('Matthew', 'Brown', 6, 6, 0)
Values in line are out of range: ('Andrew', 'Jones', 0, 3, 5)
Values in line are out of range: ('Jessica', 'Taylor', 7, 6, 11)
Excited people count: 5
Most similar to Sandra Thompson: Betty Thomas


# 6 🌈

In computers, colors can be described in different formats, but usually using three values: R (red), G (green) and B (blue), in that order. One color format would be three integers from 0 to 255, and another - a hexadecimal number starting with a hashtag.

E.g. both of these variables define the intense red color:
```python
red_tuple = (255, 0, 0)
red_hex = "#FF0000"
```
And these are for the yellow color, which is a mix of red and green:
```python
yellow_tuple = (255, 255, 0)
yellow_hex = "#FFFF00"
```
So, in the hex color, first two letters are for red, next two letters - for green, and last two - for blue. Hexadecimal colors always have the same length: #XXXXXX. Hexadecimal 255 is "FF", and 0 is "00".

Write a function named `convert_color()` to convert one color format into another. The function should automatically detect whether it received the first or the second format. Then, it should output the result in the other format. So, input `(255, 0, 0)` should return `"#FF0000"`, and `"#FF0000"` should return `(255, 0, 0)`. In case of incorrect input, raise a custom `InvalidColor` exception with a message of your choice.
- Consider these approaches to convert between decimal and hexadecimal numbers: `hex(_)`, `f"{_:x}"`, `int(_, 16)`.
- The function should not be case-sensitive (`'#abcdef'` == `'#ABCDEF'`).

In [81]:
class InvalidColor(Exception):
  def __init__(self, message):
    self.message = message
    super().__init__(self.message)

# your code
def convert_rgb_to_hex(rgb: tuple[int, int, int]) -> str:
    hex_format ='#'
    if len(rgb) != 3:
        raise InvalidColor(f"Invalid length of RGB tuple. {len(rgb)=}")
    for rgb_code in rgb:
        if not isinstance(rgb_code, int):
            raise TypeError("RGB codes must be int!")

        if not (0 <= rgb_code <= 255):
            raise InvalidColor(f"Invalid RGB code: {rgb_code}. Mut be 0–255!")
        hex_format += f"{rgb_code:X}" if rgb_code > 15 else f"0{rgb_code:X}"
    return hex_format

def convert_hex_to_rgb( hex: str) ->  tuple[int, int, int]:
    if len(hex) !=7:
        raise InvalidColor(f"Invalid hex string length: {len(hex)=}")
    if not all( char in string.hexdigits for char in hex[1:] ):
        raise InvalidColor("Wrong chars in hex format!")
    return ( int(hex[1:3], 16), int(hex[3:5], 16), int(hex[5:], 16) )

def convert_color(
        color: tuple[int, int, int] | str
    ) -> tuple[int, int, int] | str:
    if isinstance(color, str):
        return convert_hex_to_rgb(color)
    elif isinstance(color, tuple):
        return convert_rgb_to_hex(color)
    else:
        raise InvalidColor("Invalid input! Should be tuple or hex-string.")

In [82]:
assert convert_color((0, 255, 0)) == "#00FF00"
assert convert_color((0, 255, 255)) == "#00FFFF"
assert convert_color((100, 200, 150)) == "#64C896"
assert convert_color("#0000FF") == (0, 0, 255)
assert convert_color("#ABCDEF") == (171, 205, 239)